In [1]:
# Travelling Salesman Problem - notoriously NP-complete
from qiskit_optimization.applications import Tsp

In [3]:
# Generating a graph of 3 nodes
n = 3
num_qubits = n ** 2
tsp = Tsp.create_random_instance(n, seed=123)
print(tsp)

In [4]:
qp = tsp.to_quadratic_program()
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: TSP

Minimize
 obj: [ 96 x_0_0*x_1_1 + 96 x_0_0*x_1_2 + 182 x_0_0*x_2_1 + 182 x_0_0*x_2_2
      + 96 x_0_1*x_1_0 + 96 x_0_1*x_1_2 + 182 x_0_1*x_2_0 + 182 x_0_1*x_2_2
      + 96 x_0_2*x_1_0 + 96 x_0_2*x_1_1 + 182 x_0_2*x_2_0 + 182 x_0_2*x_2_1
      + 126 x_1_0*x_2_1 + 126 x_1_0*x_2_2 + 126 x_1_1*x_2_0 + 126 x_1_1*x_2_2
      + 126 x_1_2*x_2_0 + 126 x_1_2*x_2_1 ]/2
Subject To
 c0: x_0_0 + x_0_1 + x_0_2 = 1
 c1: x_1_0 + x_1_1 + x_1_2 = 1
 c2: x_2_0 + x_2_1 + x_2_2 = 1
 c3: x_0_0 + x_1_0 + x_2_0 = 1
 c4: x_0_1 + x_1_1 + x_2_1 = 1
 c5: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End



In [5]:
from qiskit_optimization.converters import QuadraticProgramToQubo

qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))

Offset: 7581.0
Ising Hamiltonian:
-1290.0 * ZIIIIIIII
- 1290.0 * IZIIIIIII
+ 606.5 * ZZIIIIIII
- 1290.0 * IIZIIIIII
+ 606.5 * ZIZIIIIII
+ 606.5 * IZZIIIIII
- 1268.5 * IIIZIIIII
+ 606.5 * ZIIZIIIII
+ 15.75 * IZIZIIIII
+ 15.75 * IIZZIIIII
- 1268.5 * IIIIZIIII
+ 15.75 * ZIIIZIIII
+ 606.5 * IZIIZIIII
+ 15.75 * IIZIZIIII
+ 606.5 * IIIZZIIII
- 1268.5 * IIIIIZIII
+ 15.75 * ZIIIIZIII
+ 15.75 * IZIIIZIII
+ 606.5 * IIZIIZIII
+ 606.5 * IIIZIZIII
+ 606.5 * IIIIZZIII
- 1282.5 * IIIIIIZII
+ 606.5 * ZIIIIIZII
+ 22.75 * IZIIIIZII
+ 22.75 * IIZIIIZII
+ 606.5 * IIIZIIZII
+ 12.0 * IIIIZIZII
+ 12.0 * IIIIIZZII
- 1282.5 * IIIIIIIZI
+ 22.75 * ZIIIIIIZI
+ 606.5 * IZIIIIIZI
+ 22.75 * IIZIIIIZI
+ 12.0 * IIIZIIIZI
+ 606.5 * IIIIZIIZI
+ 12.0 * IIIIIZIZI
+ 606.5 * IIIIIIZZI
- 1282.5 * IIIIIIIIZ
+ 22.75 * ZIIIIIIIZ
+ 22.75 * IZIIIIIIZ
+ 606.5 * IIZIIIIIZ
+ 12.0 * IIIZIIIIZ
+ 12.0 * IIIIZIIIZ
+ 606.5 * IIIIIZIIZ
+ 606.5 * IIIIIIZIZ
+ 606.5 * IIIIIIIZZ


In [10]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
import networkx as nx

In [8]:
# solving Quadratic Program using exact classical eigensolver
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

result = exact.solve(qubo)
print(result)

optimal function value: 202.0
optimal value: [1. 0. 0. 0. 1. 0. 0. 0. 1.]
status: SUCCESS


In [11]:
# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver()
result = ee.compute_minimum_eigenvalue(qubitOp)
adj_matrix = nx.to_numpy_matrix(tsp.graph)

print("energy:", result.eigenvalue.real)
print("tsp objective:", result.eigenvalue.real + offset)
x = tsp.sample_most_likely(result.eigenstate)
print("feasible:", qubo.is_feasible(x))
z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))

energy: -7379.0
tsp objective: 202.0
feasible: True
solution: [0, 1, 2]
solution objective: 202.0


In [14]:
# Run on QC
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit import Aer
algorithm_globals.random_seed = 123
seed = 10598
backend = Aer.get_backend("aer_simulator_statevector")
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

In [17]:
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
spsa = SPSA(maxiter=300)
ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
vqe = VQE(ry, optimizer=spsa, quantum_instance=quantum_instance)

result = vqe.compute_minimum_eigenvalue(qubitOp)

print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
x = tsp.sample_most_likely(result.eigenstate)
print("feasible:", qubo.is_feasible(x))
z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))

energy: -7326.024699521837
time: 4.787964582443237
feasible: True
solution: [1, 2, 0]
solution objective: 202.0


In [18]:
algorithm_globals.random_seed = 123
seed = 10598
backend = Aer.get_backend("aer_simulator_statevector")
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

In [19]:
# create minimum eigen optimizer based on VQE
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
vqe_optimizer = MinimumEigenOptimizer(vqe)

# solve quadratic program
result = vqe_optimizer.solve(qp)
print(result)

z = tsp.interpret(x)
print("solution:", z)
print("solution objective:", tsp.tsp_value(z, adj_matrix))


optimal function value: 202.0
optimal value: [0. 0. 1. 1. 0. 0. 0. 1. 0.]
status: SUCCESS
solution: [1, 2, 0]
solution objective: 202.0
